In [ ]:
import re
import sys
import warnings
from collections import Counter, defaultdict
from federated_eval_helper_functions import *
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.stats.api as sms
from matplotlib import pyplot as plt
from pylab import cm
from scipy.stats import ttest_ind
from sklearn import set_config
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import statistics
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    FunctionTransformer,
    LabelBinarizer,
    LabelEncoder,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
import pickle

In [ ]:
print(sys.executable)
print(sys.version)
print(sys.version_info)

# Load Data

# Preparing data

In [ ]:
silos = []
for i in range(9):
    silos.append(eval("silo" + str(i + 1)))

In [ ]:
int_cols = [
    "IDADE_MATERNA",
    "PESO_INICIAL",
    "IMC",
    "NUMERO_CONSULTAS_PRE_NATAL",
    "IDADE_GESTACIONAL_ADMISSAO",
    "SEMANAS_GESTACAO_PARTO",
    "PESO_ADMISSAO_INTERNAMENTO"


]
%store int_cols

In [ ]:
drop_cols = [
    "IDENTIFICADOR",
    "G_TERAPEUTICA",
    "DATA_PARTO",
    "CESARIANAS_MOTIVO_ANTERIOR",
    "NUM_RN",
    "E_ALT_UT",
      'HIPERTENSAO_CRONICA',
         'HIPERTENSAO_GESTACIONAL',
         'HIPERTENSAO_PRE_ECLAMPSIA',
         'DIABETES_GESTACIONAL_DIETA',
         'DIABETES_GESTACIONAL_INSULINA',
         'DIABETES_GESTACIONAL_ANTIBIO',
         'DIABETES_MATERNA',
         'DIABETES_TIPO1',
         'DIABETES_TIPO2',
         'HEMATOLOGICA',
         'RESPIRATORIA',
         'CEREBRAL',
         'CARDIACA',
    "ESTIMATIVA_PESO_ECO_24",
    "ESTIMATIVA_PESO_ECO_25",
    "ESTIMATIVA_PESO_ECO_26",
    "ESTIMATIVA_PESO_ECO_27",
    "ESTIMATIVA_PESO_ECO_28",
    "ESTIMATIVA_PESO_ECO_29",
    "ESTIMATIVA_PESO_ECO_30",
    "ESTIMATIVA_PESO_ECO_31",
    "ESTIMATIVA_PESO_ECO_32",
    "ESTIMATIVA_PESO_ECO_33",
    "ESTIMATIVA_PESO_ECO_34",
    "ESTIMATIVA_PESO_ECO_35",
    "ESTIMATIVA_PESO_ECO_36",
    "ESTIMATIVA_PESO_ECO_37",
    "ESTIMATIVA_PESO_ECO_38",
    "ESTIMATIVA_PESO_ECO_39",
    "ESTIMATIVA_PESO_ECO_40",
    "ESTIMATIVA_PESO_ECO_41",
    "ESTIMATIVA_PESO_ECO_42",
    "APRESENTACAO_24",
    "APRESENTACAO_25",
    "APRESENTACAO_26",
    "APRESENTACAO_27",
    "APRESENTACAO_28",
    "APRESENTACAO_29",
    "APRESENTACAO_30",
    "APRESENTACAO_31",
    "APRESENTACAO_32",
    "APRESENTACAO_33",
    "APRESENTACAO_34",
    "APRESENTACAO_35",
    "APRESENTACAO_36",
    "APRESENTACAO_37",
    "APRESENTACAO_38",
    "APRESENTACAO_39",
    "APRESENTACAO_40",
    "APRESENTACAO_41",
    "APRESENTACAO_42",
"ALCOOL",
"ESTUPEFACIENTES"
]

In [ ]:
cat_cols = list(set(silos[0].columns) - set(drop_cols) - set(int_cols))
%store cat_cols

In [ ]:
silos_null=[]
for silo in silos:
    
    silos_null.extend(silo.columns[silo.isna().all()].tolist())
Counter(silos_null) 

In [ ]:
high_null=[]
for silo in silos:
         for col in silo.columns:
            if sum(silo[col].isna())/len(silo)>0.9:
                high_null.append(col)
for k,v in Counter(high_null).items():
    if v>8:
        print(k)

In [ ]:
def create_pipeline_with_y(
    data,
    target,
    int_cols,
    cat_cols,
    manual_encoder,
    drop_cols=None,
    pipeline=None,
    op="encoder",
    as_df=False,
):
    _cat_cols = [colc for colc in cat_cols if colc not in [target]]
    _int_cols = [coli for coli in int_cols if coli not in [target]]
    n_df = data.copy().drop(columns=drop_cols + [target])

    def mapping_enc(x,encoder):
        #print(x,encoder)
        return encoder[x]
    
    vfunc = np.vectorize(mapping_enc)

    def manual_ordinal_encoder(x):
        for idx,c in enumerate(_cat_cols):
         #   print(manual_encoder[c])
            print(c)
            x[:,idx]=vfunc(x[:,idx],manual_encoder[c])

        return x

    def to_object(x):
        return pd.DataFrame(x).astype(str)

    def to_number(x):
        return pd.DataFrame(x).astype(float)

    if pipeline == None:
        fun_str = FunctionTransformer(to_object)
        fun_num = FunctionTransformer(to_number)
        unified_ordinal_enconder = FunctionTransformer(manual_ordinal_encoder)

        numeric_transformer = Pipeline(
            steps=[                ("fun_num", fun_num),
                ("imputer1", SimpleImputer(strategy="median", missing_values=np.nan)),
                ("imputer2", SimpleImputer(strategy="median", missing_values=-1)),
            ]
        )

        categorical_transformer = Pipeline(
            steps=[
             #   ("fun_str", fun_str),
                (
                    "imputer",
                    SimpleImputer(
                        missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
                    ),
                ),
                ("ordinalEncoder", unified_ordinal_enconder),
            ]
        )

        preprocessor = ColumnTransformer(
            transformers=[
                ("num", numeric_transformer, _int_cols),
                ("cat", categorical_transformer, _cat_cols),
            ]
        )

        pipeline = Pipeline(steps=[("preprocessor", preprocessor)])

        y = data[target]
        X = n_df
        y_pipe = make_pipeline(
            SimpleImputer(
                missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
            )
        )
        yy, l = create_target(y, op=op)
        XX = pipeline.fit_transform(X)

        if as_df == True:
            col_list = (
                pipeline["preprocessor"].transformers_[0][2] #changes col location
                + pipeline["preprocessor"].transformers_[1][2]
            )
            print(len(col_list))
            df1 = pd.DataFrame(XX, columns=col_list)
            r = pd.concat([df1, pd.DataFrame(yy)], axis=1)
            r.rename(columns={0: target}, inplace=True)
            return r, None, pipeline, l
        return XX, yy, pipeline, l

In [ ]:
def preprocessing_df(
    df,
    categorical_columns,
    integ_columns,
    drop_cols=["G_TERAPEUTICA", "IDENTIFICADOR"],
    pipeline=["imputer", "encoder"],
):
    if len(df.columns) != len(categorical_columns) + len(integ_columns) + len(drop_cols):
        # raise Exception
        raise("WARNING! Columns number different")
    df1 = df.copy()

    def simple_imputer_categorical_df(df, columns):
        imputer = SimpleImputer(
            missing_values=np.nan, strategy="constant", fill_value="NULLIMP"
        )  # está a por NULL em variaveis com nrs(apesar de considerar cat)
        imputer.fit(df[columns])
        X = pd.DataFrame(imputer.transform(df[columns]))
        return X

    def simple_imputer_numeric_df(df, columns):
        imputer = SimpleImputer(missing_values=np.nan, strategy="median")
        imputer.fit(df[columns].astype(float))
        X = pd.DataFrame(imputer.transform(df[columns]))
        return X

    def scikit_one_hot_encoder(df, categorical_columns):
        enc_hot = OneHotEncoder(handle_unknown="ignore", sparse=False)
        enc_hot.fit(df[categorical_columns].astype(str))
        X = pd.DataFrame(
            enc_hot.transform(df[categorical_columns].astype(str)),
            columns=enc_hot.get_feature_names(df[categorical_columns].columns),
        )
        data_encoded = pd.concat([df, X], axis=1)
        data_encoded.drop(columns=df[categorical_columns].columns, inplace=True)
        return data_encoded

    df1.drop(columns=drop_cols, inplace=True)
    if "imputer" in pipeline:
        df1[categorical_columns] = simple_imputer_categorical_df(
            df1, categorical_columns
        )
        df1[integ_columns] = simple_imputer_numeric_df(df1, integ_columns)

    if "encoder" in pipeline:
        df1 = scikit_one_hot_encoder(df1, categorical_columns)

    return df1

In [ ]:
#transform all from SIM/NAO to S/N and similar
unique_values={}
standard_map={"ADEQUADA":"A","LIMITE":"L","DESCONHECIDO,RH_DESCONHECIDO":np.nan,"DESCONHECIDO,":np.nan,"SIM":"S","NAO":"N","-1":np.nan,-1:np.nan,"NS":np.nan,",":np.nan,"Sim":"S","UNKNOWN":np.nan,"Desconhecida":np.nan,"Desconhecido":np.nan,"DESCONHECIDO":np.nan," ":np.nan,"Desconhecido,":np.nan,"  ":np.nan}
def standardize_null(x,mapping):
    if x in mapping.keys():
        return mapping[x]
    if pd.isna(x):
        return np.nan
    return x
for silo in silos:
    for col in silo.columns:
        silo[col]=silo[col].apply(standardize_null,mapping=standard_map)
        
    for col in silo.columns:
        if col in cat_cols:
            if col in unique_values.keys():
                unique_values[col].extend(silo[col].unique())
            else:
                unique_values[col]=list(silo[col].unique())
for k,v in unique_values.items():
    unique_values[k]=["NULLIMP" if  pd.isnull(x) else x for x in unique_values[k]]
    unique_values[k]=set(unique_values[k])
manual_encoder={}
for k,v in unique_values.items():
    n=len(v)
    manual_encoder[k]=dict(zip(v,[str(x) for x in (range(0,n))]))
manual_encoder
#with open('manual_encoder.json', 'w') as outfile:
# write
with open('manual_encoder.pkl', 'wb') as f:
    pickle.dump(manual_encoder, f)

In [ ]:
silosprep = []
silo_imputed = []
silos_stats = []
for silo in silos:
    
    silos_stats.append(silo.copy())
    silosprep.append(
        preprocessing_df(
            silo,
            categorical_columns=cat_cols,
            integ_columns=int_cols,
            drop_cols=drop_cols,
        )
    )
    silo_imputed.append(
        create_pipeline_with_y(
            silo.copy(),
            "GRUPO_ROBSON",
            int_cols=int_cols,
            unique_values=unique_values,
            cat_cols=cat_cols,
            as_df=True,
            drop_cols=drop_cols, 
        )[0]
    )

In [ ]:
resulting_list = []
for silo in silosprep:
    resulting_list.extend(x for x in silo.columns.tolist() if x not in resulting_list)
for col in resulting_list:
    for i in range(len(silosprep)):
        if col not in silosprep[i].columns:
            silosprep[i][col] = 0

In [ ]:
c = Counter()
final_list = []
for s in silos_stats:
    c.update(s.columns)
for col in c.keys():
    if c[col] == 9:
        final_list.append(col)

In [ ]:
for col in final_list:
    for i in range(len(silos_stats)):
        #  print(silos_stats[i].columns)
        if col not in silos_stats[i].columns:
            silos_stats[i][col] = np.nan

In [ ]:
for silo in silos_stats:
    print(len(silo.columns))